# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [27]:
# Import Python packages 
import pandas as pd
import cassandra
from cassandra.query import dict_factory
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/chloedeng/PycharmProjects/Udacity/Project 2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [8]:
def insert_data(query, session, values, dtypes, file):
    """
    Pipeline to insert data to a Apache Casssandra table from CSV file    
    Args:
        query (str): INSERT statement query
        session (object): Apache Casssandra session
        values (list): list of indicies to pass to INSERT statement
        dtypes (list): list of types to convert `values`
        file (str): CSV file path   
    Returns:
        None
    """
    assert(len(values) ==  len(dtypes))
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple([dtype(line[x]) for x, dtype in zip(values, dtypes)]))

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Answer to Q1: 
###### (1) First to create a new table called 'music_history' to store artist, song title, song's length, sessionId, and itemInSession in one table. 
###### (2) Select the columns to be inserted into the 'music_history' table. 
###### (3) Write the qeury to check the records in 'music_history' that has sessionId=338 and itemInSession=4. 

In [61]:
query = """CREATE TABLE IF NOT EXISTS music_history """
query = query + """(sessionId int, itemInSession int,
                    artist text, song_title text, length Decimal, 
                    PRIMARY KEY (sessionId,itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [62]:
# filename
file = 'event_datafile_new.csv'
# query
query = "INSERT INTO music_history (sessionId, itemInSession, artist, song_title, length)"
query = query + " VALUES (%s, %s, %s, %s, %s)"
# values and data types
values = [8, 3, 0, 9, 5]
dtypes = [int, int, str, str, float]
# insert to table
insert_data(query, session, values, dtypes, file)

#### Do a SELECT to verify that the data have been inserted into each table

In [82]:
query = "select artist, song_title, length from music_history WHERE sessionId=338 and itemInSession=4;"
try:
    session.row_factory = dict_factory
    rows = session.execute(query)
except Exception as e:
    print(e)

output=[]
for row in rows:
    output.append(row)
music_history_output=pd.DataFrame(output)

#### The query output returns one row, in the csv file, there is only one record that has sessionId = 338 and itemInSession = 4

In [83]:
music_history_output

,artist,length,song_title
0,Faithless,495.3073,Music Matters (Mark Knight Dub)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Answer to Q2:
##### (1) Create a new table called 'song_playlist_session' that has artist, song tile, itemInSession, firstName, lastName, userid, sesssionId in a table. SInce we will be using userid and sessionid to query the data, these two columns should be the primary key pair. 
##### (2) Select the proper columns of data from the csv files to be loaded to the 'song_playlist_session' table. 
#### (3) Write a query to check the records that has userid=10 and sessionid=182. 

In [20]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + """
(userid int, sessionid int, song_title text, artist text, itemInSession int, firstName text, lastName text,
PRIMARY KEY ((userid,sessionid), song_title, artist))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [23]:
# filename
file = 'event_datafile_new.csv'
# query
query = "INSERT INTO song_playlist_session (userid, sessionid, song_title, artist, itemInSession, firstName, lastName)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
# values and data types
values = [10, 8, 9, 0, 3, 1, 4]
dtypes = [int, int, str, str, int, str, str]
# insert to table
insert_data(query, session, values, dtypes, file)

In [80]:
query = """select artist, song_title, firstName, lastName 
            from song_playlist_session WHERE userid=10 and sessionid=182;"""
try:
    session.row_factory = dict_factory
    rows = session.execute(query)
except Exception as e:
    print(e)

output=[]
for row in rows:
    output.append(row)
song_playlist_session_output=pd.DataFrame(output)

#### The query output shows that there are four songs that have userid = 10 and sessionid = 182, which means the user Sylvie Cruz listened to four different songs during session 182. 

In [81]:
song_playlist_session_output

,artist,firstname,lastname,song_title
0,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...
1,Three Drives,Sylvie,Cruz,Greece 2000
2,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On
3,Sebastien Tellier,Sylvie,Cruz,Kilometer


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Answer to Q3:
##### (1) Create a new table called 'user_playlist_history' to store the firstName, lastName, song_title from the csv file. Given we are going to use song_title to query the records, song_title should be the primary key for the table. Also, since the data size is large, we would want to partition the data by the unique userid column. 
##### (2) Select data from above columns from the raw file and load them to the user_playlist_history table. 
##### (3) Check the record by creating a select query to check record for song title 'All Hands Against His Own'

In [85]:
query = "CREATE TABLE IF NOT EXISTS user_playlist_history "
query = query + """
(song_title text, userid int, firstName text, lastName text, PRIMARY KEY ((song_title),userid))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)     

In [87]:
# filename
file = 'event_datafile_new.csv'
# query
query = "INSERT INTO user_playlist_history (song_title, userid, firstName, lastName)"
query = query + " VALUES (%s, %s, %s, %s)"
# values and data types
values = [9,10,1,4]
dtypes = [str, int, str, str]
# insert to table
insert_data(query, session, values, dtypes, file)

In [90]:
query = """select firstname, lastname
            from user_playlist_history WHERE song_title='All Hands Against His Own';"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
output=[]
for row in rows:
    output.append(row)
user_playlist_history_output=pd.DataFrame(output)

#### The query output shows that there are four people that has song 'All Hands Against His Own' in their playlist. 

In [91]:
user_playlist_history_output

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [92]:
query = "drop table music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_playlist_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [93]:
session.shutdown()
cluster.shutdown()